### Most popular movie

| USER ID | MOVIE_ID | RATINGS | TIMESTAMP |
|---|---|---|---|
|196|	242|	3|	881250949|



In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster('local').setAppName('popular movie')

In [4]:
# use it when run as script
#sc = SparkContext(conf=conf)

sc = SparkContext.getOrCreate()
sc.appName="popular movie"
sc

<SparkContext master=local[*] appName=popular movie>

In [11]:
#load data
data = sc.textFile('./datasets/ml-100k/u.data')

In [34]:
movies = data.map(lambda x : (int(x.split()[1]),1)).reduceByKey(lambda x,y : x+y)

popular_movies = movies.sortBy(lambda x : x[1],ascending=False)
popular_movies.collect()

[(50, 583),
 (258, 509),
 (100, 508),
 (181, 507),
 (294, 485),
 (286, 481),
 (288, 478),
 (1, 452),
 (300, 431),
 (121, 429),
 (174, 420),
 (127, 413),
 (56, 394),
 (7, 392),
 (98, 390),
 (237, 384),
 (117, 378),
 (172, 367),
 (222, 365),
 (204, 350),
 (313, 350),
 (405, 344),
 (79, 336),
 (210, 331),
 (151, 326),
 (173, 324),
 (69, 321),
 (748, 316),
 (168, 316),
 (269, 315),
 (257, 303),
 (195, 301),
 (423, 300),
 (9, 299),
 (318, 298),
 (276, 298),
 (302, 297),
 (22, 297),
 (328, 295),
 (96, 295),
 (118, 293),
 (15, 293),
 (25, 293),
 (183, 291),
 (216, 290),
 (176, 284),
 (64, 283),
 (234, 280),
 (202, 280),
 (28, 276),
 (191, 276),
 (89, 275),
 (111, 272),
 (275, 268),
 (12, 267),
 (742, 267),
 (357, 264),
 (82, 261),
 (135, 259),
 (289, 259),
 (238, 256),
 (97, 256),
 (268, 255),
 (546, 254),
 (196, 251),
 (186, 251),
 (70, 251),
 (333, 251),
 (475, 250),
 (153, 247),
 (132, 246),
 (228, 244),
 (125, 244),
 (144, 243),
 (483, 243),
 (194, 241),
 (245, 240),
 (323, 240),
 (197, 2

In [37]:
#print name of movie rather than id
movie_data = sc.textFile("./datasets/ml-100k/u.item")

In [39]:
movie_data.sample(False,0.001).collect()

['909|Dangerous Beauty (1998)|23-Jan-1998||http://us.imdb.com/M/title-exact?imdb-title-118892|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '1018|Tie Me Up! Tie Me Down! (1990)|01-Jan-1990||http://us.imdb.com/Title?%A1%C1tame%21+(1990)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0']

we can read the movies as dict and replace the ids with names on the driver or spark automagically forwards all python objects refrenced in map will be forwarded to respective nodes, however this can be costlier if the table loaded is huge, so we can tell spark to send the data only once to the nodes using broadcast
    

In [44]:
def loadMovieNames():
    movies = movie_data.map(lambda x: (int(x.split("|")[0]), x.split("|")[1])) #(909,Dangerous Beauty (1998))
    return movies.collectAsMap()

In [67]:
# broadcast the object
movies_dict=sc.broadcast(loadMovieNames())

In [68]:
# popular movies - count the number of viewers
movies = data.map(lambda x: (int(x.split()[1]),1)) #(909, 1)
movie_freq = movies.reduceByKey(lambda x,y : x+y ) # 909 - (1,1) = (909 , 2)

In [72]:
popular_movies = movie_freq.sortBy(keyfunc=lambda x: x[1],ascending=False)
# use .value to read values which are broadcasted
popular_movies = popular_movies.map(lambda movie : (movies_dict.value[movie[0]], movie[1])) 
popular_movies.sample(False,0.001).collect() # load a few rather than all 

[('Lost in Space (1998)', 18),
 ('Midnight Dancers (Sibak) (1994)', 5),
 ('Very Natural Thing, A (1974)', 1)]